In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

plt.style.use('science')

In [2]:
df = pd.read_csv("../data/results-benchmark.csv")

In [3]:
df.head()

,n,idx,distribution,travel,serv,cost_profile,dotsp,enum,lns,mtsp,nnsvf,svf,tsp,lnsinit
0,6,0,0,0,0,"(0.5, 2.5, 10)",416.0,395.078292,395.078,447.0,423.0,473.0,416.0,395.078
1,6,0,0,0,0,"(1.0, 2.5, 10)",481.0,460.767292,460.767,512.0,501.0,573.0,481.0,460.767
2,6,0,0,0,0,"(2.0, 2.5, 10)",610.0,592.145292,592.145,644.0,658.0,773.0,610.0,592.145
3,6,0,0,0,1,"(0.5, 2.5, 10)",683.0,621.380942,621.381,734.0,687.0,694.0,683.0,621.381
4,6,0,0,0,1,"(1.0, 2.5, 10)",748.0,691.553479,692.968,805.0,781.0,794.0,748.0,692.968


In [4]:
df

,n,idx,distribution,travel,serv,cost_profile,dotsp,enum,lns,mtsp,nnsvf,svf,tsp,lnsinit
0,6,0,0,0,0,"(0.5, 2.5, 10)",416.0,395.078292,395.078,447.0,423.0,473.0,416.0,395.078
1,6,0,0,0,0,"(1.0, 2.5, 10)",481.0,460.767292,460.767,512.0,501.0,573.0,481.0,460.767
2,6,0,0,0,0,"(2.0, 2.5, 10)",610.0,592.145292,592.145,644.0,658.0,773.0,610.0,592.145
3,6,0,0,0,1,"(0.5, 2.5, 10)",683.0,621.380942,621.381,734.0,687.0,694.0,683.0,621.381
4,6,0,0,0,1,"(1.0, 2.5, 10)",748.0,691.553479,692.968,805.0,781.0,794.0,748.0,692.968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,40,19,0,0,0,"(1.0, 2.5, 10)",3752.0,NaN,3861.039,3772.0,4112.0,4684.0,3752.0,3779.289
1076,40,19,0,0,0,"(2.0, 2.5, 10)",4012.0,NaN,4072.480,4007.0,4780.0,5720.0,4012.0,4023.095
1077,40,19,0,0,1,"(0.5, 2.5, 10)",6610.0,NaN,6199.544,6632.0,6555.0,6724.0,6610.0,6228.582
1078,40,19,0,0,1,"(1.0, 2.5, 10)",6740.0,NaN,6383.993,6813.0,6969.0,7212.0,6740.0,6377.501


## Preprocess data

In [5]:
INSTANCE_COLUMNS = ['n', 'idx', 'distribution', 'travel', 'serv', 'cost_profile']
ALGORITHM_COLUMNS = [col for col in df.columns if col not in INSTANCE_COLUMNS]

df["best"] = df[ALGORITHM_COLUMNS].min(axis=1)
df = df.round(0)

In [6]:
for col in ALGORITHM_COLUMNS:
    df[col + '_gap'] = (df[col] - df['best']) / df['best'] * 100

In [7]:
# Get the weight coefficient for the travel times
df['$\omega^T$'] = df.cost_profile.str[1:4]
df['$n$'] = df['n']

# Create new gap columns with algorithm names in uppercase 
for col in [col for col in df.columns if col.endswith("_gap")]:
    alg = col.removesuffix("_gap").upper() # algorithm name in uppercase
    df[alg] = df[col]
    
ALGS = ["LNS", "MTSP", "DOTSP", "NNSVF", "LNSINIT"]

## Small instances $(n \leq 10)$

In [8]:
res = df[df.n <= 10]

# Get relevant columns and groupby
res = res.groupby(['$n$', '$\omega^T$', 'serv'])[ALGS].mean().round(2)
res = res.unstack().swaplevel(0, 1, axis=1).sort_index(axis=1)

# Fix the column ordering
order = [(0, 'LNSINIT'), (0, 'MTSP'), (0, 'DOTSP'), (0, 'NNSVF'), (1, 'LNSINIT'), (1,  'MTSP'), (1, 'DOTSP'), (1, 'NNSVF')]
res = res[order] 

res

serv                 0                          1                    
               LNSINIT  MTSP DOTSP  NNSVF LNSINIT   MTSP DOTSP  NNSVF
$n$ $\omega^T$                                                       
6   0.5           0.80  5.69  4.50  11.16    2.59  10.95  6.82  10.99
    1.0           0.75  4.83  3.45  13.17    1.96   8.15  5.65  12.70
    2.0           0.78  2.71  2.45  16.50    1.95   7.22  4.19  15.73
8   0.5           1.12  5.26  3.75   9.48    3.95  10.36  7.29   9.71
    1.0           0.85  4.47  2.88  12.17    2.85   7.91  5.68  11.39
    2.0           0.53  3.99  1.97  16.43    2.23   6.36  3.96  14.95
10  0.5           1.96  7.72  5.50  10.27    3.32  10.48  8.65  11.16
    1.0           0.88  5.86  4.09  13.71    2.62   8.55  6.82  13.64
    2.0           0.75  4.64  2.81  19.75    1.68   5.81  4.61  18.55

In [9]:
tmp = res.copy()

# Add dummy columns
tmp.insert(0, 'dum1', '')
tmp.insert(5, 'dum2', '')

print(tmp.to_latex(float_format="%.2f", escape=True))
print(res.mean().to_frame().T.to_latex(float_format="%.2f", escape=True))
res

\begin{tabular}{lllrrrrlrrrr}
\toprule
 & serv & dum1 & \multicolumn{4}{r}{0} & dum2 & \multicolumn{4}{r}{1} \\
 &  &  & LNSINIT & MTSP & DOTSP & NNSVF &  & LNSINIT & MTSP & DOTSP & NNSVF \\
$n$ & $\omega^T$ &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{6} & 0.5 &  & 0.80 & 5.69 & 4.50 & 11.16 &  & 2.59 & 10.95 & 6.82 & 10.99 \\
 & 1.0 &  & 0.75 & 4.83 & 3.45 & 13.17 &  & 1.96 & 8.15 & 5.65 & 12.70 \\
 & 2.0 &  & 0.78 & 2.71 & 2.45 & 16.50 &  & 1.95 & 7.22 & 4.19 & 15.73 \\
\cline{1-12}
\multirow[t]{3}{*}{8} & 0.5 &  & 1.12 & 5.26 & 3.75 & 9.48 &  & 3.95 & 10.36 & 7.29 & 9.71 \\
 & 1.0 &  & 0.85 & 4.47 & 2.88 & 12.17 &  & 2.85 & 7.91 & 5.68 & 11.39 \\
 & 2.0 &  & 0.53 & 3.99 & 1.97 & 16.43 &  & 2.23 & 6.36 & 3.96 & 14.95 \\
\cline{1-12}
\multirow[t]{3}{*}{10} & 0.5 &  & 1.96 & 7.72 & 5.50 & 10.27 &  & 3.32 & 10.48 & 8.65 & 11.16 \\
 & 1.0 &  & 0.88 & 5.86 & 4.09 & 13.71 &  & 2.62 & 8.55 & 6.82 & 13.64 \\
 & 2.0 &  & 0.75 & 4.64 & 2.81 & 19.75 &  & 1.68 & 5.81 & 4.61 & 18

serv                 0                          1                    
               LNSINIT  MTSP DOTSP  NNSVF LNSINIT   MTSP DOTSP  NNSVF
$n$ $\omega^T$                                                       
6   0.5           0.80  5.69  4.50  11.16    2.59  10.95  6.82  10.99
    1.0           0.75  4.83  3.45  13.17    1.96   8.15  5.65  12.70
    2.0           0.78  2.71  2.45  16.50    1.95   7.22  4.19  15.73
8   0.5           1.12  5.26  3.75   9.48    3.95  10.36  7.29   9.71
    1.0           0.85  4.47  2.88  12.17    2.85   7.91  5.68  11.39
    2.0           0.53  3.99  1.97  16.43    2.23   6.36  3.96  14.95
10  0.5           1.96  7.72  5.50  10.27    3.32  10.48  8.65  11.16
    1.0           0.88  5.86  4.09  13.71    2.62   8.55  6.82  13.64
    2.0           0.75  4.64  2.81  19.75    1.68   5.81  4.61  18.55

In [10]:
res.mean().reset_index().groupby("level_1").mean()

,serv,0
level_1,,
DOTSP,0.5,4.726111
LNSINIT,0.5,1.753889
MTSP,0.5,6.720000
NNSVF,0.5,13.414444


## Large instances $(n \ge 15)$

In [11]:
res = df[df.n >= 15]

# Get relevant columns and groupby
res = res.groupby(['$n$', '$\omega^T$', 'serv'])[ALGS].mean().round(2)
res = res.unstack().swaplevel(0, 1, axis=1).sort_index(axis=1)

# Fix the column ordering
order=[(0, 'LNSINIT'), (0, 'MTSP'), (0, 'DOTSP'), (0, 'NNSVF'), (1, 'LNSINIT'), (1,  'MTSP'), (1, 'DOTSP'), (1, 'NNSVF')]
res = res[order] 

res

serv                 0                          1                   
               LNSINIT  MTSP DOTSP  NNSVF LNSINIT  MTSP DOTSP  NNSVF
$n$ $\omega^T$                                                      
15  0.5           0.24  3.99  3.09   9.19    0.13  6.22  4.55   6.86
    1.0           0.31  2.65  2.23  13.80    0.22  4.10  3.32  10.19
    2.0           0.40  1.95  1.77  22.24    0.35  3.04  2.44  17.29
20  0.5           0.30  3.88  2.76   9.02    0.29  6.23  5.43   7.45
    1.0           0.46  3.21  2.24  14.23    0.16  4.30  4.31  11.16
    2.0           0.50  1.87  1.53  23.12    0.29  3.16  3.18  18.62
25  0.5           0.42  2.75  2.28   8.62    0.21  6.36  5.80   7.66
    1.0           0.45  1.49  1.56  13.66    0.35  5.04  4.85  11.59
    2.0           0.29  1.57  0.93  22.90    0.63  3.57  3.21  18.71
30  0.5           0.31  3.49  2.01   7.49    0.32  6.29  5.87   6.46
    1.0           0.30  2.40  1.14  12.55    0.41  5.85  4.72   9.99
    2.0           0.22  1.49  0.70  22.54    0.18  4.34  3.13  16.89
35  0.5           0.17  2.45  1.50   7.31    0.35  6.36  5.44   6.74
    1.0           0.27  1.71  0.79  12.28    0.23  5.25  4.51  10.61
    2.0           0.25  1.65  0.57  22.16    0.41  4.15  3.35  18.24
40  0.5           0.11  2.31  0.97   6.53    0.23  7.45  6.35   6.94
    1.0           0.28  1.82  0.47  11.72    0.49  6.42  5.22  10.51
    2.0           0.43  1.23  0.25  21.70    0.20  4.82  3.47  17.40

In [12]:
tmp = res.copy()

# Add dummy columns
tmp.insert(0, 'dum1', '')
tmp.insert(5, 'dum2', '')

print(tmp.to_latex(float_format="%.2f", escape=True))
print(res.mean().to_frame().T.to_latex(float_format="%.2f", escape=True))
res

\begin{tabular}{lllrrrrlrrrr}
\toprule
 & serv & dum1 & \multicolumn{4}{r}{0} & dum2 & \multicolumn{4}{r}{1} \\
 &  &  & LNSINIT & MTSP & DOTSP & NNSVF &  & LNSINIT & MTSP & DOTSP & NNSVF \\
$n$ & $\omega^T$ &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{15} & 0.5 &  & 0.24 & 3.99 & 3.09 & 9.19 &  & 0.13 & 6.22 & 4.55 & 6.86 \\
 & 1.0 &  & 0.31 & 2.65 & 2.23 & 13.80 &  & 0.22 & 4.10 & 3.32 & 10.19 \\
 & 2.0 &  & 0.40 & 1.95 & 1.77 & 22.24 &  & 0.35 & 3.04 & 2.44 & 17.29 \\
\cline{1-12}
\multirow[t]{3}{*}{20} & 0.5 &  & 0.30 & 3.88 & 2.76 & 9.02 &  & 0.29 & 6.23 & 5.43 & 7.45 \\
 & 1.0 &  & 0.46 & 3.21 & 2.24 & 14.23 &  & 0.16 & 4.30 & 4.31 & 11.16 \\
 & 2.0 &  & 0.50 & 1.87 & 1.53 & 23.12 &  & 0.29 & 3.16 & 3.18 & 18.62 \\
\cline{1-12}
\multirow[t]{3}{*}{25} & 0.5 &  & 0.42 & 2.75 & 2.28 & 8.62 &  & 0.21 & 6.36 & 5.80 & 7.66 \\
 & 1.0 &  & 0.45 & 1.49 & 1.56 & 13.66 &  & 0.35 & 5.04 & 4.85 & 11.59 \\
 & 2.0 &  & 0.29 & 1.57 & 0.93 & 22.90 &  & 0.63 & 3.57 & 3.21 & 18.71 \

serv                 0                          1                   
               LNSINIT  MTSP DOTSP  NNSVF LNSINIT  MTSP DOTSP  NNSVF
$n$ $\omega^T$                                                      
15  0.5           0.24  3.99  3.09   9.19    0.13  6.22  4.55   6.86
    1.0           0.31  2.65  2.23  13.80    0.22  4.10  3.32  10.19
    2.0           0.40  1.95  1.77  22.24    0.35  3.04  2.44  17.29
20  0.5           0.30  3.88  2.76   9.02    0.29  6.23  5.43   7.45
    1.0           0.46  3.21  2.24  14.23    0.16  4.30  4.31  11.16
    2.0           0.50  1.87  1.53  23.12    0.29  3.16  3.18  18.62
25  0.5           0.42  2.75  2.28   8.62    0.21  6.36  5.80   7.66
    1.0           0.45  1.49  1.56  13.66    0.35  5.04  4.85  11.59
    2.0           0.29  1.57  0.93  22.90    0.63  3.57  3.21  18.71
30  0.5           0.31  3.49  2.01   7.49    0.32  6.29  5.87   6.46
    1.0           0.30  2.40  1.14  12.55    0.41  5.85  4.72   9.99
    2.0           0.22  1.49  0.70  22.54    0.18  4.34  3.13  16.89
35  0.5           0.17  2.45  1.50   7.31    0.35  6.36  5.44   6.74
    1.0           0.27  1.71  0.79  12.28    0.23  5.25  4.51  10.61
    2.0           0.25  1.65  0.57  22.16    0.41  4.15  3.35  18.24
40  0.5           0.11  2.31  0.97   6.53    0.23  7.45  6.35   6.94
    1.0           0.28  1.82  0.47  11.72    0.49  6.42  5.22  10.51
    2.0           0.43  1.23  0.25  21.70    0.20  4.82  3.47  17.40

In [13]:
res.mean().reset_index().groupby("level_1").mean()

,serv,0
level_1,,
DOTSP,0.5,2.942778
LNSINIT,0.5,0.310000
MTSP,0.5,3.746111
NNSVF,0.5,13.176944
